<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Ejemplo de un python Dashboard

v1.1

In [ ]:
%%sh
pip install -q dash
pip install -q dash_core_components
pip install -q dash_html_components
pip install -q dash_table
pip install chart-studio

In [ ]:
%%writefile dash_app_1.py

import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__)#, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True, port=8050)

Overwriting dash_app_1.py


In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8050)"))

https://23rbx8o7id4-496ff2e9c6d22116-8050-colab.googleusercontent.com/


In [ ]:
!python dash_app_1.py

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "dash_app_1" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [ ]:
%%writefile dash_app_2.py

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash import Dash
from dash.dependencies import Input, Output

from sklearn import datasets
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

import chart_studio.plotly as py
from plotly.graph_objs import *

app = Dash(__name__)
server = app.server
app.title='Iris Data Dashboard'

# Leer el dataset y separar los datos en X e y
iris = datasets.load_iris()
colors = ["#E41A1C", "#377EB8", "#4DAF4A", \
          "#984EA3", "#FF7F00", "#FFFF33", \
          "#A65628", "#F781BF", "#999999"]
number_of_clusters = range(9)


df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target
col = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
X = df[col]
y = df["target"]

df_list = []

for n in number_of_clusters:
    kmeans = KMeans(n_clusters=n + 1)

    # fit data to features
    kmeans.fit(X)

    results_df = pd.DataFrame(iris.data, columns=iris.feature_names)
    results_df["predicted_classes"] = kmeans.labels_

    num_of_clusters = results_df['predicted_classes'].nunique()
    df_list.append({
        'results_df': results_df,
        'num_of_clusters': num_of_clusters
    })

dropdown_val = iris.feature_names

app.layout = html.Div(className="container", children=[
                html.Div(className="jumbotron text-center", children=[
                    html.H1("Iris Analysis"),
                    html.P("Select the X and Y to visualize the data using chosen number of clusters"),
                    html.P("A simple classification algorithm using K Nearest Neighbors")
                    ]),
                dcc.Dropdown(className="col-md-4", id="dropdown_x",
                    options=[
                        {'label': val, 'value': val} for val in dropdown_val
                    ],
                    value=dropdown_val[0] # valor por defecto para el primer lanzamiento
                ),
                dcc.Dropdown(className="col-md-4", id="dropdown_y",
                    options=[
                        {'label': val, 'value': val} for val in dropdown_val
                    ],
                    value=dropdown_val[1]
                ),
                dcc.Dropdown(className="col-md-4", id="dropdown_k",
                    options=[
                        {'label': str(val+1), 'value': val} for val in number_of_clusters
                    ],
                    value=number_of_clusters[3]
                ),
                html.Div(style={"padding": "20px"}, children=[
                    dcc.Graph(id="cluster")
                ])
    ])

# importar librerias externas css
app.css.append_css({"external_url": "https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css"})

# importar librerias externas javascript
app.scripts.append_script({"external_url": "https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js"})

@app.callback(Output("cluster", "figure"), [Input('dropdown_x', 'value'),Input('dropdown_y', 'value'),Input('dropdown_k', 'value')])
def update_graph(x, y, k):
    data = []
    num_of_clusters = df_list[k]['num_of_clusters']
    for i in range(num_of_clusters):
        results_df = df_list[k]['results_df']
        cluster_df = results_df[results_df["predicted_classes"] == i]
        data.append({
            "x": cluster_df[x],
            "y": cluster_df[y],
            "type": "scatter",
            "mode": "markers",
            "name": f"class_{i}",
            "marker": dict(
                color = colors[i],
                size = 10
            )
        })
        
    layout = {
        "hovermode": "closest", 
        "margin": {
            "r": 10, 
            "t": 25, 
            "b": 40, 
            "l": 60
        }, 
        "title": f"Iris Dataset - {x} vs {y}", 
        "xaxis": {
            "domain": [0, 1], 
            "title": x
        }, 
        "yaxis": {
            "domain": [0, 1], 
            "title": y
        }
    }

    fig = Figure(data=data, layout=layout)
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)

Writing dash_app_2.py


In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8051)"))

https://i4ysze6u0p8-496ff2e9c6d22116-8051-colab.googleusercontent.com/


In [ ]:
!python dash_app_2.py

Dash is running on http://127.0.0.1:8051/

 * Serving Flask app "dash_app_2" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
/usr/local/lib/python3.6/dist-packages/dash/resources.py:68: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plot.com/external-resources

/usr/local/lib/python3.6/dist-packages/dash/resources.py:68: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plot.com/exte